In [ ]:
import numpy as np
import tensorflow as tf
import scipy
import cnn as model
import config
import input_data
import os

In [ ]:
DATA_PATH = os.path.join(os.getcwd(),"Data")
input_image = scipy.io.loadmat(os.path.join(DATA_PATH, config.dataset_file_name))[config.dataset_mat_name]
label_image = scipy.io.loadmat(os.path.join(DATA_PATH, config.dataset_gt_file_name))[config.dataset_gt_mat_name]


height = label_image.shape[0]
width = label_image.shape[1]
channels = input_image.shape[2]


PATCH_SIZE = config.patch_size
class_num = config.num_classes
batch_size = 32


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
print(height)
print(width)
print(channels)
print(input_image.shape)
print(label_image.shape)

In [ ]:
## Scaling Down the image to 0 - 1
input_image = np.pad(input_image, ((PATCH_SIZE//2, PATCH_SIZE//2), (PATCH_SIZE//2, PATCH_SIZE//2), (0, 0)), 'symmetric')
input_image = input_image.astype(float)
input_image -= np.min(input_image)
input_image /= np.max(input_image)

In [ ]:
def mean_array(data):
    mean_arr = data.reshape(data.shape[0], data.shape[1]*data.shape[2])
    mean_arr = np.mean(mean_arr, axis = 1)
    return np.array(mean_arr)

In [ ]:
def Patch(data,height_index,width_index):
    #to [channel,size,size]
    transpose_array = data.transpose((2,0,1))

    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = transpose_array[:, height_slice, width_slice]

    mean = mean_array(transpose_array)
    
    mean_patch = np.subtract(patch, mean.reshape(-1,1,1))

    return mean_patch

In [ ]:
def placeholder_inputs():
    images_placeholder = tf.placeholder(tf.float32, shape=(None, model.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(None))
    is_training = tf.placeholder(dtype=tf.bool)
    return images_placeholder, labels_placeholder, is_training

In [ ]:
def fill_feed_dict(data_set, batch_size, images_pl, labels_pl, is_training, TF):
    images_feed, labels_feed = data_set.next_batch(batch_size)
    feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
      is_training: TF,
    }
    return feed_dict, labels_feed

In [ ]:
def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            is_training,
            data_set):

    predict_image = np.zeros((height,width))
    
    # run one epoch of eval.
    steps_per_epoch = data_set.num_examples // batch_size
    num_rest_examples = data_set.num_examples - steps_per_epoch * batch_size
    total = 0
    for step in range(steps_per_epoch):
        feed_dict, labels_feed = fill_feed_dict(data_set,
                                   batch_size,
                                   images_placeholder,
                                   labels_placeholder,
                                   is_training,
                                   False,)
        prediction = sess.run(eval_correct, feed_dict=feed_dict)

        for i in range(batch_size):
            predict_image[total//width][total%width] = prediction[i]
            total = total + 1
    
    if ( num_rest_examples != 0 ):
        feed_dict, labels_feed = fill_feed_dict(data_set,
                                   num_rest_examples,
                                   images_placeholder,
                                   labels_placeholder,
                                   is_training,
                                   False,)
        prediction = sess.run(eval_correct, feed_dict=feed_dict)
        for i in range(num_rest_examples):
            predict_image[total//width][total%width] = prediction[i]
            total = total + 1
            
    return predict_image
 


In [ ]:
def calculate_precision(matrix):
    matrix_sum_col = np.zeros(class_num)
    matrix_sum_row = np.zeros(class_num)
    diagonal_sum = 0
    total_num = 0
    for i in range(class_num):
        diagonal_sum += matrix[i][i]
        for j in range(class_num):
            matrix_sum_col[j] += matrix[i][j]
            matrix_sum_row[i] += matrix[i][j]
            total_num += matrix[i][j]

    oa = diagonal_sum / total_num
    aa = 0
    print('class#\tright\ttest\trate')
    for i in range(class_num):
        prec_per_class = matrix[i][i] / matrix_sum_row[i]
        aa += prec_per_class
        print('%d\t%d\t%d\t%.5f' % (i+1, int(matrix[i][i]), int(matrix_sum_row[i]),prec_per_class))
    aa = aa / class_num

    kappa_temp = 0
    for i in range(class_num):
        kappa_temp += matrix_sum_col[i] * matrix_sum_row[i]
    kappa = (total_num*diagonal_sum-kappa_temp) / (total_num*total_num-kappa_temp)
   
    print('  OA: %.5f' % oa)
    print('  AA: %.5f' % aa)
    print('  kappa: %.5f' % kappa)


In [ ]:
def create_datasets():
    
    image_patch = []
    
    for i in range(height):
        for j in range(width):
            image_patch.extend(Patch(input_image,i,j))

    
    image_patch = np.array(image_patch)
    image_patch = image_patch.reshape(-1, channels, PATCH_SIZE, PATCH_SIZE)
    
    target = label_image.flatten()
    target = np.transpose(target)

    return image_patch, target


In [ ]:
def decoder():
    
    model_name = DATA_PATH + '/record/' + config.dataset + '/cnn_t0-7X7.ckpt-299'
                               
    with tf.Graph().as_default():
        
        images_placeholder, labels_placeholder, is_training = placeholder_inputs()

        logits = model.inference(images_placeholder, is_training)
        
        predict = model.predicting(logits)
        
        saver = tf.train.Saver()

        sess = tf.Session()

        saver.restore(sess,model_name)

                    
        predict_image = do_eval(sess,
                                predict,
                                images_placeholder,
                                labels_placeholder,
                                is_training,
                                datasets)

        return predict_image

In [ ]:

image_patch, target = create_datasets()


In [ ]:
# del datasets
datasets = input_data.DataSet(image_patch, target)
predict_image = decoder()

In [ ]:
matrix = np.zeros((class_num,class_num))
label_image = label_image.astype(int)
predict_image = predict_image.astype(int)
for i in range(height):
    for j in range(width):
        if(label_image[i][j] != 0):
            matrix[label_image[i][j]-1][predict_image[i][j]] += 1

calculate_precision(matrix)

In [ ]:
cmap = np.asarray( [[0, 0, 0],               
                      [255, 0, 0],         
                      [0, 255, 0],       
                      [0, 0, 255],
                      [255, 255, 0],
                      [255, 0, 255],
                      [0, 255, 255],
                      [0, 170, 170],
                      [170, 0, 170],
                      [170, 170, 0],
                      [127, 0, 255],
                      [0, 127, 255],
                      [255, 127, 0],
                      [127, 0, 0],
                      [0, 127, 0],
                      [0, 0, 127],
                      [127, 127, 127]], dtype='int32')

# cmap = np.asarray( [[0, 0, 0],               
#                        [255, 0, 0],         
#                        [0, 255, 0],       
#                        [0, 0, 255],
#                        [255, 255, 0],
#                        [255, 0, 255],
#                        [0, 255, 255],
#                        [0, 170, 170],
#                        [170, 0, 170],
#                        [170, 170, 0]], dtype='int32')
y_rgb = cmap[predict_image.astype(int)+1, :]
y_image = y_rgb.reshape(height, width, 3)
scipy.misc.imsave(DATA_PATH + '/record/' + config.dataset + '/cnn.png' , y_image)

y_rgb = cmap[label_image.astype(int), :]
y_image = y_rgb.reshape(height, width, 3)
scipy.misc.imsave(DATA_PATH + '/record/' + config.dataset + '/label.png' , y_image)